# DWPC computation using neo4j

In [ ]:
import threading
import concurrent.futures
import gzip
import csv
import time

import pandas
import py2neo

import hetio.readwrite
import hetio.neo4j

In [ ]:
# Read hetnet metagraph
path = 'https://raw.githubusercontent.com/dhimmel/integrate/c56d312a53e351d26016d5fd4751a9715cc3f0c3/data/metagraph.json'
metagraph = hetio.readwrite.read_metagraph(path)

In [ ]:
# Extract metapaths
metapaths = metagraph.extract_metapaths('compound', 'disease', max_length=4)
metapaths.pop(0)
metapath_to_query = {str(metapath): hetio.neo4j.construct_dwpc_query(metapath, 'identifier')
                     for metapath in metapaths}
len(metapaths)

In [ ]:
# Read drug-compound disease pairs
part_df = pandas.read_table('data/partition.tsv.gz')
part_df.tail()

In [ ]:
# Take a subset of compounds and diseases to reduce runtime
disease_df = part_df.groupby(['disease_id', 'disease_name']).indication.sum().reset_index()
compound_df = part_df.groupby(['compound_id', 'compound_name']).indication.sum().reset_index()
disease_df = disease_df.query('indication >= 10')
compound_df = compound_df.query('indication >= 3')
part_df = part_df[part_df.compound_id.isin(compound_df.compound_id) & part_df.disease_id.isin(disease_df.disease_id)]
len(disease_df), len(compound_df), len(part_df)

In [ ]:
# Open neo4j connection
uri = 'http://localhost:7474/db/data/'
neo = py2neo.Graph(uri)

In [ ]:
def generate_parameters(max_elems=None):
    """Generate compound, disease, metapath combinations"""
    n = 0
    for i, row in part_df.iterrows():
        for metapath, query in metapath_to_query.items():
            if max_elems is not None and n == max_elems:
                break
            yield {
                'compound_id': row['compound_id'],
                'disease_id': row['disease_id'],
                'metapath': metapath,
                'query': query,
                'w': 0.4,
            }
            n += 1

In [ ]:
def compute_dwpc(query, metapath, compound_id, disease_id, w):
    """Execute the neo4j query and write results to file"""
    start = time.time()
    results = neo.cypher.execute(query, source=compound_id, target=disease_id, w=w)
    record = results.one
    seconds = '{0:.4g}'.format(time.time() - start)
    row = compound_id, disease_id, metapath, record['PC'], w, record['DWPC'], seconds
    with writer_lock:
        writer.writerow(row)

In [ ]:
%%time

# Parameters
workers = 16
max_elems = None

# Prepare writer
path = 'data/dwpc.tsv.gz'
write_file = gzip.open(path, 'wt')
writer = csv.writer(write_file, delimiter='\t')
writer.writerow(['compound_id', 'disease_id', 'metapath', 'PC', 'w', 'DWPC', 'seconds'])

# Create ThreadPoolExecutor
executor = concurrent.futures.ThreadPoolExecutor(max_workers=workers)
writer_lock = threading.Lock()

# Submit jobs
for params in generate_parameters(max_elems):
    while executor._work_queue.qsize() > 5000:
        time.sleep(1)
    executor.submit(compute_dwpc, **params)

# Shutdown and close
executor.shutdown()
write_file.close()